In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-fepj4pq5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-fepj4pq5
  Resolved https://github.com/huggingface/transformers.git to commit 348e2294ac1a3cc744a93f04ab1f886439ee90d3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import librosa

audio_file_path = '/content/drive/MyDrive/CHI24_DHHTranscription/Lecture Videos/PrelimNoTranscription.mp3'
audio_array, sampling_rate = librosa.load(audio_file_path, sr=16000)

In [ ]:
audio_data = {
    "array": audio_array,
    "sampling_rate": sampling_rate
}

audio_data, len(audio_data['array'])


({'array': array([ 1.2113204e-05,  2.7984519e-05,  2.4546298e-05, ...,
         -6.6963057e-06, -4.1230032e-06,  1.9476872e-06], dtype=float32),
  'sampling_rate': 16000},
 20367035)

In [ ]:
import torch
from datasets import load_dataset

# #### "openai/whisper-small"
# from transformers import WhisperProcessor, WhisperForConditionalGeneration
# processor = WhisperProcessor.from_pretrained("openai/whisper-small")
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
# model.config.forced_decoder_ids = None

#### pipeline method
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device)

model_id = "openai/whisper-small"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    return_timestamps=False,
    torch_dtype=torch_dtype,
    device=device,
)

    # max_new_tokens=128,
    # chunk_length_s=30,
    # batch_size=16,

In [ ]:
# ## whisper-small
# input_features = processor(audio_data["array"], sampling_rate=audio_data["sampling_rate"], return_tensors="pt").input_features
# # generate token ids
# predicted_ids = model.generate(input_features)
# # decode token ids to text
# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
# transcription


## pipeline method
transcription = pipe(audio_data)
transcription

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


{'text': " obvious is what is a cryptocurrency? So this word was kind of invented 10 years ago when, I don't know how many of you know the origin story of where Bitcoin came from, but basically a pseudonym on the internet dropped a paper and some open source code in a forum on an email list and said, hey, I have this idea for this thing called Bitcoin. It's kind of like electronic cash. Here's how I think it could work. And here is some code if you want to run it and become part of this peer-to-peer network. We don't know who this person is. This person is basically virtually disappeared from the internet and from the world. But it's created something that has captured so many people's imaginations and has sort of, depending on how you measure it, created billions and billions of dollars of economic value and sort of inspired a lot of people to think about how to use this technology to solve a myriad of different problems, not just electronic payments. So cryptocurrencies and the techn

In [ ]:
import json

exDict = {'exDict': transcription}

with open('/content/drive/MyDrive/CHI24_DHHTranscription/Lecture Videos/PrelimTranscript.txt', 'w') as file:
     file.write(json.dumps(exDict))


In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
# result = pipe(sample)
# print(result["text"])


In [ ]:
dataset[0]["audio"]

{'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de',
 'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
        0.0005188 ]),
 'sampling_rate': 16000}

In [ ]:
result = pipe(sample, return_timestamps=True)
print(result["chunks"])

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


[{'timestamp': (0.0, 5.28), 'text': ' Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.'}, {'timestamp': (6.34, 10.1), 'text': " Nor is Mr. Quilter's manner less interesting than his matter."}, {'timestamp': (10.92, 17.6), 'text': ' He tells us that at this festive season of the year, with Christmas and roast beef looming before us,'}, {'timestamp': (18.44, 22.58), 'text': ' similes drawn from eating and its results occur most readily to the mind.'}, {'timestamp': (23.16, 28.66), 'text': " He has grave doubts whether Sir Frederick Leighton's work is really Greek after all,"}, {'timestamp': (29.1, 32.48), 'text': ' and can discover in it but little of rocky Ithaca.'}, {'timestamp': (33.62, 37.86), 'text': " Linnell's pictures are a sort of Upguards and Adam paintings,"}, {'timestamp': (37.86, 42.88), 'text': " and Mason's exquisite idylls are as national as a jingo poem."}, {'timestamp': (44.56, 45.78), 'text': " Mr. Burkett Foster's"}, {'timestamp